### Importaciones

In [121]:
import pandas as pd
from datetime import datetime
import pandas as pd
import numpy as np
import random
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 
Mes -> 	Fecha del mes en formato YYYY-MM-DD. Representa el mes al que corresponden los datos climáticos.
temperature_2m_max	-> Temperatura máxima promedio del mes medida a 2 metros de altura, en grados Celsius (°C).
temperature_2m_min -> Temperatura mínima promedio del mes, también a 2 metros de altura, en grados Celsius (°C).
precipitation_sum -> Precipitación total del mes en milímetros (mm). Suma de toda el agua caída durante ese mes.
winddirection_10m_dominant -> Dirección dominante del viento a 10 metros de altura, en grados. Se mide en sentido azimutal desde el norte. (Ej.: 0° = norte, 90° = este, etc.).
SEREMI -> Nombre de la región sanitaria (SEREMI) a la que corresponde esta fila.


### Creación de DataSet's de clima mensual por año y SEREMI:

In [122]:
# 1. Cargar todos los archivos seremi
seremi_data = {
    "2014": pd.read_csv("seremi_2014.csv"),
    "2015": pd.read_csv("seremi_2015.csv"),
    "2016": pd.read_csv("seremi_2016.csv"),
    "2017": pd.read_csv("seremi_2017.csv"),
    "2018": pd.read_csv("seremi_2018.csv"),
    "2019": pd.read_csv("seremi_2019.csv"),
    "2020": pd.read_csv("seremi_2020.csv"),
    "2021": pd.read_csv("seremi_2021.csv"),
    "2022": pd.read_csv("seremi_2022.csv"),
    "2023": pd.read_csv("seremi_2023.csv"),
    "2024": pd.read_csv("seremi_2024.csv"),
    "2025": pd.read_csv("seremi_2025.csv"),
}

# 2. Función para procesar cada dataset climático
def procesar_datasets_climaticos(seremi_data_dict):
    def grados_a_cardinal(grados):
        if pd.isna(grados):
            return None
        direcciones = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
        index = int((grados + 22.5) % 360 // 45)
        return direcciones[index]

    for año, df in seremi_data_dict.items():
        # Eliminar columna innecesaria
        if "Unnamed: 0" in df.columns:
            df = df.drop(columns=["Unnamed: 0"])

        # Crear columna cardinal
        if "winddirection_10m_dominant" in df.columns:
            df["wind_direction_cardinal"] = df["winddirection_10m_dominant"].apply(grados_a_cardinal)

        # Renombrar columnas
        df = df.rename(columns={
            "temperature_2m_max": "Temperatura Máxima",
            "temperature_2m_min": "Temperatura Mínima",
            "precipitation_sum": "Precipitaciones (suma)",
            "winddirection_10m_dominant": "Dirección Viento"
        })

        # Limpiar columna de grados si existe y renombrar cardinal
        if "Dirección Viento" in df.columns:
            df = df.drop(columns=["Dirección Viento"])
        if "wind_direction_cardinal" in df.columns:
            df = df.rename(columns={"wind_direction_cardinal": "Dirección del Viento"})

        # Guardar el dataframe procesado
        seremi_data_dict[año] = df

    return seremi_data_dict

# 3. Aplicar la función a todos los datasets climáticos
seremi_data = procesar_datasets_climaticos(seremi_data)

# 4. Verificar que todo esté correcto (opcional)
seremi_data["2015"].head(12)

,Mes,Temperatura Máxima,Temperatura Mínima,Precipitaciones (suma),SEREMI,Dirección del Viento
0,2015-01-01,12.277419,6.867742,33.7,Magallanes,W
1,2015-02-01,12.142857,7.100000,36.9,Magallanes,W
2,2015-03-01,11.561290,6.393548,92.1,Magallanes,W
3,2015-04-01,9.093333,4.446667,57.9,Magallanes,W
4,2015-05-01,5.935484,2.541935,100.0,Magallanes,S
5,2015-06-01,3.943333,1.150000,92.8,Magallanes,SW
6,2015-07-01,3.032258,-0.138710,98.4,Magallanes,SW
7,2015-08-01,4.180645,0.509677,59.8,Magallanes,SW
8,2015-09-01,5.266667,1.120000,46.0,Magallanes,SW
9,2015-10-01,8.925806,2.803226,24.2,Magallanes,SW


### Creación de DataSet's sobre ocupación de camas por mes y por año según hospital:

In [123]:
# Archivos filtrados hospitalarios
archivos_filtrados = {
    "2014": "2014_filtrado.csv",
    "2015": "2015_filtrado.csv",
    "2016": "2016_filtrado.csv",
    "2017": "2017_filtrado.csv",
    "2018": "2018_filtrado.csv",
    "2019": "2019_filtrado.csv",
    "2020": "2020_filtrado.csv",
    "2021": "2021_filtrado.csv",
    "2022": "2022_filtrado.csv",
    "2023": "2023_filtrado.csv",
    "2024": "2024_filtrado.csv",
    "2025": "2025_filtrado.csv"
}

# Diccionario para guardar los dataframes ya limpios
filtrado_limpio = {}

# Abreviaturas de los meses
meses_abreviados = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                    'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

# Columnas que se eliminarán si existen
columnas_a_eliminar = ["Cód. SS/SEREMI", "Cód. Estab.", "Cód. Nivel Cuidado", "Acum", "Año"]

# Procesamiento de los archivos
for año, archivo in archivos_filtrados.items():
    # Cargar CSV
    df = pd.read_csv(archivo)

    # Eliminar filas irrelevantes
    df = df[~df["Nombre Nivel Cuidado"].isin([
      "330 - Area Pensionado",
      "330 - Área Pensionado",
      "401 - Área Médica Adulto Cuidados Básicos",
      "402 - Área Médica Adulto Cuidados Medios",
      "403 - Área Médico-Quirúrgico Cuidados Básicos",
      "404 - Área Médico-Quirúrgico Cuidados Medios",
      "405 - Área Cuidados Intensivos Adultos",
      "406 - Área Cuidados Intermedios Adultos",
      "407 - Área Médica Pediátrica Cuidados Básicos",
      "408 - Área Médica Pediátrica Cuidados Medios",
      "409 - Área Médico-Quirúrgico Pediátrica Cuidados Básicos",
      "410 - Área Médico-Quirúrgico Pediátrica Cuidados Medios",
      "411 - Área Cuidados Intensivos Pediátricos",
      "412 - Área Cuidados Intermedios Pediátricos",
      "413 - Área Neonatología Cuidados Básicos",
      "414 - Área Neonatología Cuidados Intensivos",
      "415 - Área Neonatología Cuidados Intermedios",
      "416 - Área Obstetricia",
      "418 - Área Psiquiatría Adulto Corta estadía",
      "419 - Área Psiquiatría Adulto Mediana estadía",
      "420 - Área Psiquiatría Adulto Larga estadía",
      "421 - Área Psiquiatría Infanto-adolescente corta estadía",
      "427 - Área Sociosanitaria Adulto",
      "428 - Área de Hospitalización de Cuidados Intensivos en Psiquiatría Adulto",
      "429 - Área de Hospitalización de Cuidados Intensivos en Psiquiatría Infanto Adolescente",
      "Área Cuidados Intensivos Adultos",
      "Área Cuidados Intensivos Pediátricos",
      "Área Cuidados Intermedios Adultos",
      "Área Cuidados Intermedios Pediátricos",
      "Área Médica Adulto Cuidados Básicos",
      "Área Médica Adulto Cuidados Medios",
      "Área Médica Pediátrica Cuidados Básicos",
      "Área Médica Pediátrica Cuidados Medios",
      "Área Médico-Quirúrgico Cuidados Básicos",
      "Área Médico-Quirúrgico Cuidados Medios",
      "Área Médico-Quirúrgico Pediátrica Cuidados Básicos",
      "Área Médico-Quirúrgico Pediátrica Cuidados Medios",
      "Área Neonatología Cuidados Básicos",
      "Área Neonatología Cuidados Intensivos",
      "Área Neonatología Cuidados Intermedios",
      "Área Obstetricia",
      "Área Pensionado",
      "Área Psiquiatría Adulto Corta estadía",
      "Área Psiquiatría Adulto Larga estadía",
      "Área Psiquiatría Adulto Mediana estadía",
      "Área Psiquiatría Infanto-adolescente corta estadía"
    ]
    )].copy()

    # Eliminar columnas innecesarias
    df = df.drop(columns=columnas_a_eliminar, errors='ignore')

    # Renombrar columnas clave
    df = df.rename(columns={
        "Nombre SS/SEREMI": "SEREMI",
        "Nombre Nivel Cuidado": "Area"
    })

    # Renombrar columnas de meses abreviados por fecha
    nuevo_nombre_columnas = {}
    año_int = int(año)

    for i, mes_abrev in enumerate(meses_abreviados, start=1):
        if mes_abrev in df.columns:
            fecha = datetime(año_int, i, 1).strftime('%Y-%m-%d')
            nuevo_nombre_columnas[mes_abrev] = fecha

    df = df.rename(columns=nuevo_nombre_columnas)

    # Guardar DataFrame limpio
    filtrado_limpio[año] = df

# Ejemplo: verificar que todo cargó bien
filtrado_limpio["2020"].head(5)

,SEREMI,Nombre Establecimiento,Area,Glosa,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01
0,Datos País,Datos País,Datos País,Dias Cama Disponibles,800179.00,745007.0,789700.00,745965.0,802598.00,795021.0,832573.00,827810.00,779335.00,805056.00,776759.00,793770.00
1,Datos País,Datos País,Datos País,Dias Cama Ocupados,627256.00,577307.0,540914.00,412450.0,477211.00,498858.0,545163.00,557324.00,540777.00,567446.00,559618.00,552408.00
2,Datos País,Datos País,Datos País,Dias de Estada,627876.00,557377.0,602302.00,406231.0,439175.00,503875.0,548854.00,515769.00,527973.00,554252.00,532859.00,589847.00
3,Datos País,Datos País,Datos País,Promedio Cama Disponibles,25812.23,25689.9,25474.19,24865.5,25890.26,26500.7,26857.19,26703.55,25977.83,25969.55,25891.97,25605.48
4,Datos País,Datos País,Datos País,Numero de Egresos,89177.00,80451.0,79438.00,54961.0,58717.00,59720.0,65544.00,66102.00,67836.00,71445.00,70530.00,72361.00


### Función para unión de datos:

In [124]:
def extraer_info_hospital_area_todo_el_año(hospital_nombre, filtrado_limpio, seremi_data):
    registros = []

    for año, df in filtrado_limpio.items():
        df_seremi = seremi_data[año].copy()

        df_seremi['Mes'] = pd.to_datetime(df_seremi['Mes'], errors='coerce')
        df_seremi.dropna(subset=['Mes'], inplace=True)
        df_seremi['Mes'] = df_seremi['Mes'].dt.to_period("M").dt.to_timestamp()

        columnas_mes = [col for col in df.columns if col.startswith("20")]
        for col in columnas_mes:
            df_tmp = df[
                (df["Nombre Establecimiento"] == hospital_nombre) &
                (df["Area"] == "Datos Establecimiento")
            ][["SEREMI", "Nombre Establecimiento", "Area", "Glosa", col]].copy()

            if df_tmp.empty:
                continue

            df_tmp = df_tmp.rename(columns={col: "Valor"})
            df_tmp["Fecha"] = pd.to_datetime(col)

            clima_mes = df_seremi[df_seremi["Mes"] == df_tmp["Fecha"].iloc[0]].copy()
            df_merge = df_tmp.merge(clima_mes, on=["SEREMI"], how="left")

            registros.append(df_merge)

    if registros:
        return pd.concat(registros, ignore_index=True).sort_values("Fecha")
    else:
        print("⚠️ No se encontraron datos para ese hospital y área.")
        return pd.DataFrame()


ejemplo_df_hospital = extraer_info_hospital_area_todo_el_año(
    "Hospital Barros Luco Trudeau (Santiago, San Miguel)",
    filtrado_limpio,
    seremi_data
)

ejemplo_df_hospital.head()

,SEREMI,Nombre Establecimiento,Area,Glosa,Valor,Fecha,Mes,Temperatura Máxima,Temperatura Mínima,Precipitaciones (suma),Dirección del Viento
0,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",Datos Establecimiento,Dias Cama Disponibles,21236.00,2014-01-01,2014-01-01,28.964516,15.748387,0.4,SW
1,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",Datos Establecimiento,Dias Cama Ocupados,20034.00,2014-01-01,2014-01-01,28.964516,15.748387,0.4,SW
2,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",Datos Establecimiento,Dias de Estada,19132.00,2014-01-01,2014-01-01,28.964516,15.748387,0.4,SW
3,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",Datos Establecimiento,Promedio Cama Disponibles,685.03,2014-01-01,2014-01-01,28.964516,15.748387,0.4,SW
4,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",Datos Establecimiento,Numero de Egresos,2311.00,2014-01-01,2014-01-01,28.964516,15.748387,0.4,SW


### Preparación de datos pre modelación:

In [125]:
def preparar_dataset_climatico_avanzado(df_datos):
    if 'Area' in df_datos.columns:
        df_datos = df_datos[df_datos['Area'] == "Datos Establecimiento"].copy()

    df_modelo = df_datos.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    columnas_necesarias = [
        'Dias Cama Disponibles', 'Dias Cama Ocupados',
        'Promedio Cama Disponibles', 'Numero de Egresos', 'Egresos Fallecidos'
    ]
    columnas_faltantes = [col for col in columnas_necesarias if col not in df_modelo.columns]
    if columnas_faltantes:
        print("❌ Faltan columnas en los datos pivotados:", columnas_faltantes)
        return None

    clima = df_datos.drop_duplicates(subset='Fecha')[[
        'Fecha', 'Temperatura Máxima', 'Temperatura Mínima',
        'Precipitaciones (suma)', 'Dirección del Viento'
    ]] if 'Temperatura Máxima' in df_datos.columns else pd.DataFrame()

    if not clima.empty:
        df_modelo = pd.merge(df_modelo, clima, on='Fecha', how='left')
        df_modelo = pd.get_dummies(df_modelo, columns=['Dirección del Viento'], prefix='Viento')

    df_modelo = df_modelo.sort_values('Fecha')
    df_modelo['Mes'] = df_modelo['Fecha'].dt.month
    df_modelo['Trimestre'] = df_modelo['Fecha'].dt.quarter

    df_modelo['lag_1'] = df_modelo['Dias Cama Disponibles'].shift(1)
    df_modelo['lag_2'] = df_modelo['Dias Cama Disponibles'].shift(2)
    df_modelo['lag_3'] = df_modelo['Dias Cama Disponibles'].shift(3)
    df_modelo['media_movil_3'] = df_modelo['Dias Cama Disponibles'].rolling(window=3).mean()
    df_modelo['variacion_disponibles'] = df_modelo['Dias Cama Disponibles'].diff()
    df_modelo['porcentaje_ocupacion'] = df_modelo['Dias Cama Ocupados'] / df_modelo['Dias Cama Disponibles']
    df_modelo['ocupados_media_movil'] = df_modelo['Dias Cama Ocupados'].rolling(window=3).mean()
    df_modelo['promedio_media_movil'] = df_modelo['Promedio Cama Disponibles'].rolling(window=3).mean()
    df_modelo['egresos_media_movil'] = df_modelo['Numero de Egresos'].rolling(window=3).mean()
    df_modelo['egresos_fallecidos_media_movil'] = df_modelo['Egresos Fallecidos'].rolling(window=3).mean()

    if 'Temperatura Máxima' in df_modelo.columns and 'Temperatura Mínima' in df_modelo.columns:
        df_modelo['Diferencia Térmica'] = df_modelo['Temperatura Máxima'] - df_modelo['Temperatura Mínima']
        df_modelo['temp_max_movil'] = df_modelo['Temperatura Máxima'].rolling(window=3).mean()
        df_modelo['precipitacion_movil'] = df_modelo['Precipitaciones (suma)'].rolling(window=3).mean()
        df_modelo['interaccion_ocupacion_temp'] = df_modelo['porcentaje_ocupacion'] * df_modelo['Temperatura Máxima']
        df_modelo['interaccion_precipitacion_disp'] = df_modelo['Precipitaciones (suma)'] * df_modelo['Dias Cama Disponibles']

    df_modelo['same_month_last_year'] = df_modelo['Dias Cama Disponibles'].shift(12)
    df_modelo['hist_avg_mes'] = df_modelo.groupby('Mes')['Dias Cama Disponibles'].transform('mean')

    return df_modelo.dropna().copy()





### Automatización de extración de datos y preparación de dataset - MODELO XGBOOST:

Estamos prediciendo aproximadamente el 20% de los meses más recientes del historial disponible para ese hospital y área. Por ejemplo:

Si el hospital tiene datos desde 2014 hasta 2025, son 12 años × 12 meses = 144 - 10 (meses faltantes del 2025) -> 134 meses totales.

El 20% de 134 son aprox 27 meses -> estamos prediciendo los últimos 27 meses, desde fines de 2022 hasta marzo de 2025 (en nuestro caso actual).

In [130]:
def pipeline_final_mae_reducido(nombre_hospital, filtrado_limpio, seremi_data):
    print(f"🔍 Extrayendo datos para: {nombre_hospital} | Datos Establecimiento")

    df_hospital_raw = extraer_info_hospital_area_todo_el_año(
        hospital_nombre=nombre_hospital,
        filtrado_limpio=filtrado_limpio,
        seremi_data=seremi_data
    )

    if df_hospital_raw.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo_features = preparar_dataset_climatico_avanzado(df_hospital_raw)
    print(df_modelo_features.shape)

    if df_modelo_features is None or df_modelo_features.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    columnas_features_modelo = [
        'Dias Cama Ocupados', 'Promedio Cama Disponibles', 'Numero de Egresos',
        'Mes', 'Trimestre', 'lag_1', 'lag_2', 'lag_3', 'media_movil_3',
        'porcentaje_ocupacion', 'variacion_disponibles',
        'ocupados_media_movil', 'promedio_media_movil', 'egresos_media_movil',
        'egresos_fallecidos_media_movil',
        'Temperatura Máxima', 'Temperatura Mínima', 'Precipitaciones (suma)',
        'Diferencia Térmica', 'temp_max_movil', 'precipitacion_movil',
        'same_month_last_year', 'hist_avg_mes',
        'interaccion_ocupacion_temp', 'interaccion_precipitacion_disp'
    ] + [col for col in df_modelo_features.columns if col.startswith('Viento_')]

    X_modelo = df_modelo_features[columnas_features_modelo]
    y_modelo = df_modelo_features['Dias Cama Disponibles']
    X_train, X_test, y_train, y_test = train_test_split(X_modelo, y_modelo, shuffle=False, test_size=0.2)

    print("🔧 Ajustando modelo con GridSearchCV...")
    grid_params = {
        'n_estimators': [50, 100],
        'max_depth': [2, 3],
        'learning_rate': [0.05, 0.1]
    }

    modelo_xgb = XGBRegressor(n_jobs=1, random_state=42)
    grid_search_modelo = GridSearchCV(
        estimator=modelo_xgb,
        param_grid=grid_params,
        scoring='neg_mean_absolute_error',
        cv=TimeSeriesSplit(n_splits=3),
        verbose=1
    )
    grid_search_modelo.fit(X_train, y_train)
    modelo_final = grid_search_modelo.best_estimator_

    print("📌 Mejores parámetros encontrados:", grid_search_modelo.best_params_)
    print("📊 Evaluando sobre el 20% final de los datos...")

    y_predicho = modelo_final.predict(X_test)
    mae = mean_absolute_error(y_test, y_predicho)
    rmse = mean_squared_error(y_test, y_predicho) ** 0.5
    error_porcentaje = np.mean(np.abs((y_test.values - y_predicho) / y_test.values)) * 100

    resultados_df = pd.DataFrame({
        'Fecha': df_modelo_features['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_predicho,
        'Error Absoluto': np.abs(y_test.values - y_predicho),
        'Error Porcentual (%)': np.abs((y_test.values - y_predicho) / y_test.values) * 100
    })

    print(f"✅ MAE (camas): {mae:.2f}")
    print(f"✅ RMSE (camas): {rmse:.2f}")
    print(f"📉 Error promedio porcentual: {error_porcentaje:.2f}%")

    return resultados_df




### Ejemplo de uso - MODELO XGBOOST:

In [131]:
pd.set_option("display.max_columns", None)
resultados_final = pipeline_final_mae_reducido("Hospital Barros Luco Trudeau (Santiago, San Miguel)", filtrado_limpio, seremi_data)

resultados_final

🔍 Extrayendo datos para: Hospital Barros Luco Trudeau (Santiago, San Miguel) | Datos Establecimiento
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
(122, 37)
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 679.60
✅ RMSE (camas): 1228.32
📉 Error promedio porcentual: 3.11%


,Fecha,Real,Predicho,Error Absoluto,Error Porcentual (%)
0,2023-02-01,18759.0,19594.283203,835.283203,4.452706
1,2023-03-01,21183.0,21600.634766,417.634766,1.971556
2,2023-04-01,20731.0,20724.978516,6.021484,0.029046
3,2023-05-01,21618.0,21562.519531,55.480469,0.256640
4,2023-06-01,20804.0,20826.691406,22.691406,0.109072
5,2023-07-01,21456.0,21484.648438,28.648438,0.133522
6,2023-08-01,21494.0,21424.523438,69.476562,0.323237
7,2023-09-01,21119.0,20872.548828,246.451172,1.166964
8,2023-10-01,21913.0,21575.701172,337.298828,1.539264
9,2023-11-01,21247.0,21230.496094,16.503906,0.077676


### Ejemplo de uso extendido a mas hospitales - MODELO XGBOOST:

In [87]:
# 1. Extraer todos los hospitales válidos con área "Datos Establecimiento"
def extraer_hospitales_validos(filtrado_limpio):
    hospitales = set()
    for df in filtrado_limpio.values():
        if "Nombre Establecimiento" in df.columns and "Area" in df.columns:
            validos = df[df["Area"] == "Datos Establecimiento"][["Nombre Establecimiento"]].dropna().drop_duplicates()
            for _, row in validos.iterrows():
                hospitales.add(row["Nombre Establecimiento"])
    return sorted(list(hospitales))

# 2. Obtener todos los hospitales válidos
hospitales_validos = extraer_hospitales_validos(filtrado_limpio)

# 3. Evaluar modelo para cada hospital
resultados_todos = {}

for hospital in hospitales_validos:
    try:
        resultados = pipeline_final_mae_reducido(hospital)
        if resultados is not None:
            resultados_todos[hospital] = resultados
    except:
        continue  # Silenciosamente ignora errores

# 4. Crear tabla resumen
tabla_resumen = []

for hospital, resultados in resultados_todos.items():
    if "Error Absoluto" in resultados.columns and "Error Porcentual (%)" in resultados.columns:
        mae = resultados["Error Absoluto"].mean()
        error_pct = resultados["Error Porcentual (%)"].mean()
        tabla_resumen.append({
            "Hospital": hospital,
            "MAE (camas)": round(mae, 2),
            "Error Porcentual Promedio (%)": round(error_pct, 2)
        })

# 5. Mostrar tabla ordenada por MAE
df_tabla_resumen = pd.DataFrame(tabla_resumen).sort_values("MAE (camas)").reset_index(drop=True)
display(df_tabla_resumen)

# 6. Calcular MAE promedio general
mae_promedio_general = df_tabla_resumen["MAE (camas)"].mean()
print(f"\n📊 MAE promedio general: {mae_promedio_general:.2f} camas")



🔍 Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia) | Datos Establecimiento
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 190.19
✅ RMSE (camas): 236.52
📉 Error promedio porcentual: 1.13%
🔍 Extrayendo datos para: Hospital Adalberto Steeger (Talagante) | Datos Establecimiento
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 79.96
✅ RMSE (camas): 88.55
📉 Error promedio porcentual: 2.31%
🔍 Extrayendo dat

/var/folders/6r/z5t8_gxs1ps8jw29h6h_q_900000gn/T/ipykernel_5598/1319329685.py:62: RuntimeWarning: divide by zero encountered in divide
  error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100
/var/folders/6r/z5t8_gxs1ps8jw29h6h_q_900000gn/T/ipykernel_5598/1319329685.py:70: RuntimeWarning: divide by zero encountered in divide
  'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100


🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 10.01
✅ RMSE (camas): 15.38
📉 Error promedio porcentual: 1.45%
🔍 Extrayendo datos para: Hospital de Tomé | Datos Establecimiento
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 69.75
✅ RMSE (camas): 83.30
📉 Error promedio porcentual: 2.54%
🔍 Extrayendo datos para: Hospital de Vilcún | Datos Establecimiento
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 

,Hospital,MAE (camas),Error Porcentual Promedio (%)
0,Hospital de Nancagua,0.00,0.00
1,Hospital de Lanco,0.01,0.00
2,Hospital de Curacaví,0.04,0.01
3,Hospital de Lolol,0.05,0.02
4,Hospital San Juan de Dios (Combarbalá),0.19,0.03
...,...,...,...
87,"Hospital Psiquiátrico El Peral (Santiago, Puen...",480.96,14.40
88,Hospital de Puerto Montt,487.11,2.52
89,Hospital Carlos Van Buren (Valparaíso),576.03,4.50
90,"Hospital Barros Luco Trudeau (Santiago, San Mi...",679.60,3.11



📊 MAE promedio general: 92.17 camas


---


### Listado y cantidad de hospitales con los que trabaja el modelo:

In [32]:
def obtener_lista_hospitales(filtrado_limpio):
    hospitales = set()
    
    for año, df in filtrado_limpio.items():
        if "Nombre Establecimiento" in df.columns:
            hospitales.update(df["Nombre Establecimiento"].dropna().unique())
    
    return sorted(list(hospitales))


In [13]:
lista_hospitales = obtener_lista_hospitales(filtrado_limpio)
print(f"Se encontraron {len(lista_hospitales)} hospitales.")
for h in lista_hospitales[:10]:  # muestra los primeros 10 como ejemplo
    print("🏥", h)


Se encontraron 92 hospitales.
🏥 Complejo Hospitalario San José (Santiago, Independencia)
🏥 Hospital Adalberto Steeger (Talagante)
🏥 Hospital Adriana Cousiño (Quintero)
🏥 Hospital Barros Luco Trudeau (Santiago, San Miguel)
🏥 Hospital Carlos Van Buren (Valparaíso)
🏥 Hospital Centro Geriátrico Paz de la Tarde (Limache)
🏥 Hospital Clorinda Avello (Santa Juana)
🏥 Hospital Clínico Herminda Martín (Chillán)
🏥 Hospital Comunitario de Salud Familiar Pedro Morales Campos (Yungay)
🏥 Hospital Comunitario de Salud Familiar de Bulnes


# 
92 hospitales comunes entre 2014 y 2025

### XGBOOST 

### PREDICCION FUTURA

### MODO ENTRENAMIENTO

In [103]:
# Función extendida para predecir camas disponibles
def predecir_camas_mes_futuro_extendida(hospital, modo="entrenamiento", mes_futuro=None):
    print(f"🔍 Extrayendo datos para: {hospital}")
    df_base = extraer_info_hospital_area_todo_el_año(hospital)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital especificado.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo = preparar_dataset_climatico_avanzado(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    todas_las_features = [
        'Dias Cama Ocupados', 'Promedio Cama Disponibles', 'Numero de Egresos',
        'Mes', 'Trimestre', 'lag_1', 'lag_2', 'lag_3', 'media_movil_3',
        'porcentaje_ocupacion', 'variacion_disponibles',
        'ocupados_media_movil', 'promedio_media_movil', 'egresos_media_movil',
        'Temperatura Máxima', 'Temperatura Mínima', 'Precipitaciones (suma)',
        'Diferencia Térmica', 'temp_max_movil', 'precipitacion_movil',
        'same_month_last_year', 'hist_avg_mes',
        'interaccion_ocupacion_temp', 'interaccion_precipitacion_disp'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    solo_features_futuras = [
        'Mes', 'Trimestre', 'Temperatura Máxima', 'Temperatura Mínima',
        'Precipitaciones (suma)', 'Diferencia Térmica',
        'same_month_last_year', 'hist_avg_mes'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    if modo == "entrenamiento":
        X = df_modelo[todas_las_features]
        y = df_modelo['Dias Cama Disponibles']
        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

        print("🔧 Ajustando modelo con GridSearchCV...")
        param_grid = {
            'n_estimators': [50, 100],
            'max_depth': [2, 3],
            'learning_rate': [0.05, 0.1]
        }

        model = XGBRegressor(n_jobs=1, random_state=42)
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring='neg_mean_absolute_error',
            cv=TimeSeriesSplit(n_splits=3),
            verbose=0
        )
        grid_search.fit(X_train, y_train)
        mejor_modelo = grid_search.best_estimator_

        print("📌 Mejores parámetros encontrados:", grid_search.best_params_)
        print("📊 Evaluando sobre el 20% final de los datos...")

        y_pred = mejor_modelo.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

        print(f"✅ MAE (camas): {mae:.2f}")
        print(f"✅ RMSE (camas): {rmse:.2f}")
        print(f"📉 Error promedio porcentual: {error_pct:.2f}%")

        resultados = pd.DataFrame({
            'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
            'Real': y_test.values,
            'Predicho': y_pred,
            'Error Absoluto': np.abs(y_test.values - y_pred),
            'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
        })

        return resultados

    elif modo == "futuro" and mes_futuro is not None:
        print(f"📅 Modo futuro: prediciendo camas para {mes_futuro}")
        X_full = df_modelo[todas_las_features]
        y_full = df_modelo['Dias Cama Disponibles']
        model = XGBRegressor(n_jobs=1, random_state=42)
        model.fit(X_full, y_full)

        df_futuro = df_modelo[df_modelo["Fecha"] == pd.to_datetime(mes_futuro)].copy()

        if df_futuro.empty:
            print(f"⚠️ No se encontraron datos climáticos o estacionales para {mes_futuro}")
            return None

        X_futuro = df_futuro[solo_features_futuras]
        prediccion = model.predict(X_futuro)[0]
        print(f"📈 Predicción para {mes_futuro}: {prediccion:.2f} camas disponibles")

        return prediccion

    else:
        print("⚠️ Modo no reconocido o mes_futuro no especificado.")
        return None




In [104]:
# Crear resultados para entrenamiento iterando hospitales
resultados_entrenamiento_todos = []

for hospital in lista_hospitales:
    try:
        print("\n==========================")
        print(f"🏥 Evaluando entrenamiento: {hospital}")
        print("==========================")

        resultado_entrenamiento = predecir_camas_mes_futuro(
            hospital=hospital,
            modo="entrenamiento"
        )

        if resultado_entrenamiento is not None:
            resultado_entrenamiento["Hospital"] = hospital
            resultados_entrenamiento_todos.append(resultado_entrenamiento)

    except Exception as e:
        print(f"❌ Error en {hospital} -> {e}")
        continue






🏥 Evaluando entrenamiento: Complejo Hospitalario San José (Santiago, Independencia)
🔍 Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia)
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 189.85
✅ RMSE (camas): 232.91
📉 Error promedio porcentual: 1.13%

🏥 Evaluando entrenamiento: Hospital Adalberto Steeger (Talagante)
🔍 Extrayendo datos para: Hospital Adalberto Steeger (Talagante)
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 79.67
✅ RMSE (camas): 88.25
📉 Error promedio porcentual: 2.30%

🏥 Evaluando entrenamiento: Hosp

/var/folders/6r/z5t8_gxs1ps8jw29h6h_q_900000gn/T/ipykernel_5598/477782436.py:59: RuntimeWarning: divide by zero encountered in divide
  error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100
/var/folders/6r/z5t8_gxs1ps8jw29h6h_q_900000gn/T/ipykernel_5598/477782436.py:70: RuntimeWarning: divide by zero encountered in divide
  'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100


🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 10.01
✅ RMSE (camas): 15.38
📉 Error promedio porcentual: 1.45%

🏥 Evaluando entrenamiento: Hospital de Tomé
🔍 Extrayendo datos para: Hospital de Tomé
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 81.98
✅ RMSE (camas): 92.17
📉 Error promedio porcentual: 2.96%

🏥 Evaluando entrenamiento: Hospital de Vilcún
🔍 Extrayendo datos para: Hospital de Vilcún
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
📌 Mejores parámetros encontrados: {'lea

In [113]:
# Crear resumen con el error porcentual promedio por hospital
errores_porcentuales = []

for df_resultado in resultados_entrenamiento_todos:
    hospital = df_resultado["Hospital"].iloc[0]
    error_pct_promedio = df_resultado["Error Porcentual (%)"].mean()
    errores_porcentuales.append({
        "Hospital": hospital,
        "Error Porcentual Promedio (%)": round(error_pct_promedio, 2)
    })

# Convertir a DataFrame y ordenar
df_errores = pd.DataFrame(errores_porcentuales).sort_values(
    by="Error Porcentual Promedio (%)", ascending=False
).reset_index(drop=True)

# Mostrar los resultados
df_errores.head(60)


,Hospital,Error Porcentual Promedio (%)
0,Hospital de Teno,inf
1,Instituto Nacional de Rehabilitación Infantil ...,56.19
2,Hospital de Curepto,49.85
3,Hospital de Llanquihue,23.76
4,Hospital de Paillaco,15.23
5,"Hospital Psiquiátrico El Peral (Santiago, Puen...",14.40
6,"Hospital Juana Ross de Edwards (Peñablanca, Vi...",13.99
7,Hospital San Juan de Dios (Curicó),13.15
8,Hospital de Río Bueno,10.96
9,Hospital de Purén,9.55
